In [10]:
import copy
import datetime
import numpy as np
import time
import pandas as pd
import pandas_datareader.data as web
from sklearn.cluster import KMeans
from sklearn import preprocessing
import plotly
import itertools
from talib.abstract import *
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='maiti.t', api_key='km8Kdfszic1Cu6ZAWiZJ')
plotly.offline.init_notebook_mode(connected=True)

In [11]:
def get_open_normalised_prices(df, start, end):
 
    df1 = pd.DataFrame(df["High"]/df["Open"])
    df1.columns=["H/O"]
    df1["L/O"] = df["Low"]/df["Open"]
    df1["C/O"] = df["Close"]/df["Open"]
 
    return df1

In [12]:
def plotgraph(dow):
# start = datetime.datetime(2015, 1, 1)
# end =  time.strftime("%Y-%m-%d")
# dow = web.DataReader("^DJI", "yahoo", start, end)
    
        plotly.offline.iplot({
            "data": [go.Candlestick(x=dow.index, open=dow['Open']/1000,
                               high=dow['High']/1000,
                               low=dow['Low']/1000,
                               close=dow['Close']/1000,name='USA')],
            "layout": Layout(title="USA - France Market Comparison ", yaxis=dict(title='Unit in USD $ '), xaxis=dict(title='Date'))
        #                     shapes= [{'x0': '2007-12-01', 'x1': '2007-12-01',
        #                                 'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
        #                                 'line': {'color': 'rgb(30,30,30)', 'width': 1}}],
        #                                 annotations= [{'x': '2007-12-01', 'y': 1, 
        #                                                  'xref': 'x','yref': 'paper',
        #                                                  'showarrow': True, 'xanchor': 'left',
        #                                                   'text': 'Official start of the recession'}])
        })

In [13]:
def create_follow_cluster_matrix(data):
 
    data["ClusterTomorrow"] = data["Cluster"].shift(-1)
    data.dropna(inplace=True)
    data["ClusterTomorrow"] = data["ClusterTomorrow"].apply(int)
    dow["ClusterMatrix"] = list(zip(data["Cluster"], data["ClusterTomorrow"]))
    cmvc = data["ClusterMatrix"].value_counts()
    clust_mat = np.zeros( (k, k) )
    for row in cmvc.iteritems():
        print(row[1]*100.0/len(data))
        clust_mat[row[0]] = row[1]*100.0/len(data)
    print("Cluster Follow-on Matrix:")
    print(clust_mat)
    return clust_mat,cmvc

In [14]:
def plotKMeanCluster(dow):
    df = dow.copy()
    df.sort_values(by="Cluster", inplace=True)
    df.reset_index(inplace=True)
    df["clust_index"] = df.index
    df["clust_change"] = df["Cluster"].diff()
    change_indices = df[df["clust_change"] != 0]
    plotly.offline.iplot({
                "data": [go.Candlestick(x=df.index, open=df['Open'],
                                   high=df['High'],
                                   low=df['Low'],
                                   close=df['Close'],name='OHLC')],
                "layout": Layout(title="K Means Cluster Analysis  ", yaxis=dict(title='Variance of OHLC in $ '), 
                                 xaxis=dict(title='Data Table Index'),
                                shapes= [{'x0': change_indices.index[1], 'x1': change_indices.index[1],
                                            'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
                                            'line': {'color': 'rgb(30,30,30)', 'width': 1}},
                                        {'x0': change_indices.index[2], 'x1': change_indices.index[2],
                                            'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
                                            'line': {'color': 'rgb(30,30,30)', 'width': 1}},
                                        {'x0': '383', 'x1': '383',
                                            'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
                                            'line': {'color': 'rgb(30,30,30)', 'width': 1}},
                                        {'x0': change_indices.index[3], 'x1': change_indices.index[3],
                                            'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
                                            'line': {'color': 'rgb(30,30,30)', 'width': 1}},
                                        {'x0': change_indices.index[4], 'x1': change_indices.index[4],
                                            'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
                                            'line': {'color': 'rgb(30,30,30)', 'width': 1}}],

                                            annotations= [{'x': change_indices.index[1], 'y': 1, 
                                                             'xref': 'x','yref': 'paper',
                                                             'showarrow': True, 'xanchor': 'left',
                                                              'text': 'Cluster 2'},
                                                         {'x': change_indices.index[2], 'y': 1, 
                                                             'xref': 'x','yref': 'paper',
                                                             'showarrow': True, 'xanchor': 'left',
                                                              'text': 'Cluster 3'},
                                                         {'x': change_indices.index[3], 'y': 1, 
                                                             'xref': 'x','yref': 'paper',
                                                             'showarrow': True, 'xanchor': 'left',
                                                              'text': 'C4'},
                                                         {'x': change_indices.index[4], 'y': 1, 
                                                             'xref': 'x','yref': 'paper',
                                                             'showarrow': True, 'xanchor': 'left',
                                                              'text': 'C5'}])
            })

In [15]:
if __name__ == "__main__":
    # Obtain DOW 30  pricing data from Yahoo Finance
    start = datetime.datetime(2005, 1, 1)
    end =  time.strftime("%Y-%m-%d")
    dow = web.DataReader("^DJI", "yahoo", start, end)
    plotgraph(dow)
 
    dow_norm = get_open_normalised_prices(dow, start, end)
    
    k = 5
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(dow_norm)
    labels = km.labels_
    dow["Cluster"] = labels
    dow_norm["Cluster"] = labels
 
    # Plot the 3D normalised candles using H/O, L/O, C/O
#     try3d(dow_norm,k)
    
    # Plot the full OHLC candles re-ordered 
#     # into their respective clusters
    plotKMeanCluster(dow)

    # Create and output the cluster follow-on matrix
    clust_mat,cmvc = create_follow_cluster_matrix(dow)

34.1624685139
14.2632241814
13.3186397985
6.80100755668
5.8564231738
5.66750629723
5.47858942065
4.59697732997
2.67632241814
0.913098236776
0.755667506297
0.66120906801
0.66120906801
0.66120906801
0.629722921914
0.44080604534
0.44080604534
0.346347607053
0.346347607053
0.283375314861
0.283375314861
0.251889168766
0.22040302267
0.157430730479
0.125944584383
Cluster Follow-on Matrix:
[[  5.6675063    0.28337531   0.66120907   4.59697733  14.26322418]
 [  0.34634761   0.15743073   0.44080605   0.34634761   0.12594458]
 [  0.66120907   0.25188917   0.22040302   0.66120907   0.91309824]
 [  5.47858942   0.44080605   0.62972292   2.67632242   5.85642317]
 [ 13.3186398    0.28337531   0.75566751   6.80100756  34.16246851]]


In [16]:
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist

In [46]:
K = range(1,15)
KM = [kmeans(dow_norm,k) for k in K]
centroids = [cent for (cent,var) in KM]   # cluster centroids
avgWithinSS = [var for (cent,var) in KM]

In [47]:
trace1 = go.Scatter(
    x = K,
    y = avgWithinSS,
    mode = 'lines+markers',
    name = 'lines+markers'
)
data = [trace1]
layout =go.Layout(title=" Bend Graph  ", yaxis=dict(title='Sum of Squares Distance '), 
                                 xaxis=dict(title='Number of features '))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='line-mode')

In [48]:
print("Plotting 3 D Graph ....")
temp = dow_norm.sort_values(by='Cluster')
data = []
for i in range(0,k):

    trace = go.Scatter3d(
        x=temp[temp['Cluster'] ==i]["H/O"],
        y=temp[temp['Cluster'] ==i]["L/O"],
        z= temp[temp['Cluster']==i]["C/O"],
        mode='markers',
        marker=dict(
            size=8,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.1,

            ),
            opacity=0.0
        )
    )

    data.append(trace)



layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0),
                   title="K Mean Clustering of OHLC",
                   scene = dict(xaxis = dict(title='HIGH/OPEN'),
                                yaxis = dict(title='LOW/OPEN'),
                                zaxis = dict(title='CLOSE/OPEN'),),
                    width=700,)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='K Means Clustering 3D')

Plotting 3 D Graph ....


In [49]:
print("Ranking of Clusters ");print(str(cmvc))
print("Centroid for the K Means Clusters ")
kmean = pd.DataFrame(km.cluster_centers_)
kmean.columns=['X','Y','Z']
print(kmean)

Ranking of Clusters 
(4, 4)    1085
(0, 4)     453
(4, 0)     423
(4, 3)     216
(3, 4)     186
(0, 0)     180
(3, 0)     174
(0, 3)     146
(3, 3)      85
(2, 4)      29
(4, 2)      24
(2, 3)      21
(2, 0)      21
(0, 2)      21
(3, 2)      20
(1, 2)      14
(3, 1)      14
(1, 3)      11
(1, 0)      11
(4, 1)       9
(0, 1)       9
(2, 1)       8
(2, 2)       7
(1, 1)       5
(1, 4)       4
Name: ClusterMatrix, dtype: int64
Centroid for the K Means Clusters 
          X         Y         Z
0  1.011521  0.998466  1.008911
1  1.003179  0.950500  0.958893
2  1.035036  0.995937  1.031867
3  1.002028  0.983354  0.987077
4  1.003292  0.995590  0.999500


In [50]:
print("Getting the Latest Trade 's OHLC Values for DOW 30 ") 
testdoeN = dow_norm.copy()
del testdoeN['Cluster']


Getting the Latest Trade 's OHLC Values for DOW 30 


In [51]:
predictedCluster = km.predict(testdoeN.tail(1).values.reshape(-1,3))[0]
print("Predicted Cluster of Given OHLC Date "+ str(predictedCluster))


Predicted Cluster of Given OHLC Date 4


In [52]:
predictedNextCluster = []
for i,row in cmvc.iteritems():
    if(i[0]==predictedCluster):
        predictedNextCluster.append(i[1])
print("Predicted Next Day Cluster would be : "+ str(predictedNextCluster))    

Predicted Next Day Cluster would be : [4, 0, 3, 2, 1]


In [53]:
probabCluster=[]
totalPrbablityfromPast=0
for i in predictedNextCluster:
        totalPrbablityfromPast=totalPrbablityfromPast+clust_mat[predictedCluster][i]
#         probabCluster.append(clust_mat[predictedCluster][i])
print("Calulating the Relative Probablity ")
for i in predictedNextCluster:
        probabCluster.append(clust_mat[predictedCluster][i]/totalPrbablityfromPast*100)

Calulating the Relative Probablity 


In [54]:
nextClusterMatrix = pd.DataFrame(predictedNextCluster)
nextClusterMatrix.columns=["Cluster"]
nextClusterMatrix['Probability']=probabCluster
print("Next Day CLuster and its Relative probablity from the Predicted Cluster : "+ str(predictedCluster))
print(nextClusterMatrix)

Next Day CLuster and its Relative probablity from the Predicted Cluster : 4
   Cluster  Probability
0        4    61.752988
1        0    24.075128
2        3    12.293682
3        2     1.365965
4        1     0.512237


In [55]:
print("Getting the filtered Past data where the maximum predicted range of the next DAY OHLC would be ")
inputClusterone = dow[dow['Cluster']==nextClusterMatrix['Cluster'][0]]
inputClustertwo = dow[dow['Cluster']==nextClusterMatrix['Cluster'][1]]

Getting the filtered Past data where the maximum predicted range of the next DAY OHLC would be 


In [56]:
plotgraph(inputClusterone)

In [28]:
plotgraph(inputClustertwo)

In [29]:
print("Datset form the first probable Cluster ")
inputClusterone.head()

Datset form the first probable Cluster 


,Open,High,Low,Close,Adj Close,Volume,Cluster,ClusterTomorrow,ClusterMatrix
Date,,,,,,,,,
2005-01-03,10783.750000,10867.389648,10710.070313,10729.429688,10729.429688,270620000,4,3,"(4, 3)"
2005-01-05,10629.530273,10684.429688,10597.750000,10597.830078,10597.830078,263550000,4,4,"(4, 4)"
2005-01-06,10593.190430,10667.580078,10589.330078,10622.879883,10622.879883,232850000,4,4,"(4, 4)"
2005-01-07,10624.799805,10653.250000,10571.740234,10603.959961,10603.959961,283770000,4,4,"(4, 4)"
2005-01-10,10603.440430,10663.740234,10582.379883,10621.030273,10621.030273,279500000,4,4,"(4, 4)"


In [44]:
import os
import sys
import tweepy
import requests
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
print("Import Function for the Neural Network Done")

Import Function for the Neural Network Done


## Open - Close Price - DOW 30 Market Indicator for last Week 

In [58]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()
from bokeh.layouts import gridplot
# from bokeh.plotting import figure, show, output_file


p1 = figure(x_axis_type="datetime", title="Stock Closing Prices")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Price'

# p1.line(pd.to_datetime(df['transactiondate'],format= '%Y-%m-%d'), df['Adj Close'], color='#A6CEE3', legend='AAPL')

p1.line(inputClusterone.head(7).index, inputClusterone['Open'].head(7), color='#A6CEE3', legend='Open')
p1.line(inputClusterone.head(7).index, inputClusterone['Close'].head(7), color='#B2DF8A', legend='Close')
show(p1)

Loading BokehJS ...

## Functions for adding best featurtes in the dataset to analyse 

In [59]:
def get_indicators(stocks, period):
#     for i in stocks:
        stocks.columns = [s.lower() for s in stocks.columns]
        print("Adding Features: Started  ")
        features = pd.DataFrame(SMA(stocks, timeperiod=10))
        features.columns = ['sma_10']
        features['NATR']=pd.DataFrame(NATR(stocks, timeperiod=14))
        features = pd.concat([features,STOCHF(stocks, fastk_period=14, fastd_period=3)], axis=1)
        features['willr'] = pd.DataFrame(WILLR(stocks, timeperiod=14))
        features['rsi'] = pd.DataFrame(RSI(stocks, timeperiod=14))
        features['wma_10'] = pd.DataFrame(WMA(stocks,10))
        features['T3'] =pd.DataFrame(T3(stocks, timeperiod=5, vfactor=0))
        features['closePrice']=pd.DataFrame(stocks['close'].shift(-period))
        features['return_pct_change'] = ROC(stocks, timeperiod=period)
        features['return_pct_change'] = features['return_pct_change'].shift(-period)
        features['pct_change'] = features['return_pct_change'].apply(lambda x: '1' if x > 0 else '0' if x <= 0 else np.nan)
        features = features.dropna()
           
        return features

In [60]:
print("Adding Features in the dataset ")
dataset1 = get_indicators(inputClusterone,1)
dataset2 = get_indicators(inputClustertwo,1)
# stock_prediction(dataset1)

Adding Features in the dataset 
Adding Features: Started  
Adding Features: Started  


In [61]:
dataset1.tail()

,sma_10,NATR,fastk,fastd,willr,rsi,wma_10,T3,closePrice,return_pct_change,pct_change
Date,,,,,,,,,,,
2017-08-07,21905.991016,0.438776,99.563139,98.764977,-0.436861,77.912689,21987.771484,21849.845639,22085.339844,-0.149559,0
2017-08-08,21953.182031,0.447444,86.270501,95.277880,-13.729499,74.653150,22020.380362,21892.697324,22048.699219,-0.165905,0
2017-08-09,21986.950977,0.444893,80.905672,88.913104,-19.094328,71.104788,22037.747124,21929.292702,21844.009766,-0.928352,0
2017-08-10,21991.696875,0.483942,50.935556,72.703910,-49.064444,55.293391,22011.757813,21951.388042,21858.320313,0.065512,1
2017-08-11,21994.497852,0.471415,46.689777,59.510335,-53.310223,56.029560,21987.507529,21960.011700,21993.710938,0.619401,1


In [62]:
print("Making The Training Dataset")
traindataset = dataset1.head(len(dataset1)-50)
traindataset2 = dataset2.head(len(dataset1)-50)

Making The Training Dataset


In [63]:
# xdataset = dataset1.copy()
# del xdataset['closePrice']

In [64]:
# y_train = np.array(dataset1['closePrice'])[2:]

In [65]:
def create_dataset(datasetReal):
    print(type(datasetReal))
#     dataset = datasetReal.copy()
#     y_train = np.array(dataset['closePrice'])
#     y_train2 = y_train[2:]
    xdataset = datasetReal.copy()
#     del xdataset['closePrice']
    xdataset = xdataset[xdataset.columns[xdataset.columns !='closePrice']]
    dataset = np.array(xdataset)
    print("Total Number of features in trainig Dataset "+ str(xdataset.columns.size))
    dataX = [dataset[n] for n in range(len(dataset)-1)]
    return np.array(dataX),xdataset.columns.size

In [66]:
def stock_prediction(trainX,trainY):
#     dataset = datasetReal.copy()
#     dataset = np.array(dataset)

    # Create dataset matrix (X=t and Y=t+1)

    
#     pd.DataFrame(dataset).to_csv("tempo.csv")
   
    

    # Create and fit Multilinear Perceptron model
    starttime= time.time()
    model = Sequential()
    model.add(Dense(5, input_dim=numbFeatures, activation='relu'))
    model.add(Dense(11, input_dim=numbFeatures, activation='relu'))
    model.add(Dense(11, input_dim=numbFeatures, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'mape'])
    history1 = model.fit(trainX, trainY, nb_epoch=20, batch_size=2, verbose=0)
    print("Time Taken to Model %s seconds ---" % (time.time() - starttime))
    print("*******************************************")
    print(history1)
    print("*******************************************")
    # Our prediction for tomorrow
#     prediction = model.predict(np.array([trainX[0],trainY[0]]))

#     result = 'The price will move from %s to %s' % (trainX[0], prediction[0][0])
    print(model.summary())

    return model,history1

In [68]:
print("Training the Neural Network for next day Prediction")
trainY = np.array(traindataset['closePrice'])[1:]
trainX, numbFeatures = create_dataset(traindataset)
m,h = stock_prediction(trainX,trainY)

trainY2 = np.array(traindataset2['closePrice'])[1:]
trainX2, numbFeatures2 = create_dataset(traindataset2)
m2 = stock_prediction(trainX2,trainY2)

Training the Neural Network for next day Prediction
<class 'pandas.core.frame.DataFrame'>
Total Number of features in trainig Dataset 10
Time Taken to Model 18.7485859394 seconds ---
*******************************************
*******************************************
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_18 (Dense)             (None, 11)                66        
_________________________________________________________________
dense_19 (Dense)             (None, 11)                132       
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 12        
Total params: 265
Trainable params: 265
Non-trainable params: 0
_________________________________________________________________
None


In [73]:
finalreport.index

RangeIndex(start=19, stop=20, step=1)

In [74]:
fullReport  = pd.DataFrame(h.history)
finalMAPE=fullReport['mean_absolute_percentage_error'].tail(1)
finalLoss = fullReport['loss'].tail(1)
finalMAE = fullReport['mean_absolute_error'].tail(1)

finalreport = pd.DataFrame(finalMAPE)
finalreport.columns = ['MAPE']
finalreport['Loss']=finalLoss
finalreport['MAE']=finalMAE
finalreport.index = [1]

print(finalreport)

     MAPE          Loss         MAE
1  1.5409  91196.435625  204.858129


In [ ]:
# min_max_scaler = preprocessing.MinMaxScaler()
# X_train_minmax = min_max_scaler.fit_transform(fullReport['mean_absolute_percentage_error'].values.reshape(-1,1))

In [ ]:
# my_list = map(lambda x: x[0], X_train_minmax)
# mapeNN = pd.Series(my_list)

In [75]:
iplot({"data": [
                  Scatter(x=fullReport.index, y=fullReport['mean_absolute_percentage_error'],name=str('MAPE')),
#                 Scatter(x=fullReport.index, y=fullReport['loss'],name=str('LOSS'))],
                Scatter(x=fullReport.index, y=fullReport['mean_absolute_error'],name=str('MAE'))],
       "layout": Layout(title="Training of Neural Network % ",
                         yaxis=dict(title='Units-->  '),
                         xaxis=dict(title='EPOCHS --> '))
})

In [76]:
iplot({"data": [Scatter(x=fullReport.index, y=fullReport['loss'],name=str('LOSS'))],
#                 Scatter(x=fullReport.index, y=fullReport['mean_absolute_error'],name=str('MAE'))],
       "layout": Layout(title="Training of Neural Network % ",
                         yaxis=dict(title='Units-->  '),
                         xaxis=dict(title='EPOCHS --> '))
})

In [77]:
trainfeature=traindataset.tail(len(traindataset)-1)
trainlabel= trainfeature['closePrice']

prediction = m.predict(trainX)
my_list = map(lambda x: x[0], prediction)
traintestlabel = pd.Series(my_list)

testdata = dataset1.tail(50)
testexpectedlabel = testdata['closePrice']


xtest = testdata[testdata.columns[testdata.columns !='closePrice']]
predictiontest = m.predict(np.array(xtest))
temp_list = map(lambda x: x[0], predictiontest)
testlabel = pd.Series(temp_list)

In [78]:
trainfeature2=traindataset2.tail(len(traindataset2)-1)
trainlabel2= trainfeature2['closePrice']

prediction2 = m.predict(trainX2)
my_list = map(lambda x: x[0], prediction2)
traintestlabel2 = pd.Series(my_list)

testdata2 = dataset2.tail(50)
testexpectedlabel2 = testdata2['closePrice']


xtest2 = testdata2[testdata2.columns[testdata2.columns !='closePrice']]
predictiontest2 = m.predict(np.array(xtest2))
temp_list2 = map(lambda x: x[0], predictiontest2)
testlabel2 = pd.Series(temp_list2)

# Visulising the data 

In [79]:


iplot({"data": [Scatter(x=trainfeature.index, y=trainlabel,name=str('Expected Train Data')),
                Scatter(x=trainfeature.index, y=traintestlabel,name=str('Actua Train Data')),
                Scatter(x=testdata.index, y=testexpectedlabel,name=str('Expected Test Data')),
                Scatter(x=testdata.index, y=testlabel,name=str('Actual')),
                Scatter(x=testdata2.index, y=testlabel2,name=str('Actual Data 2 '))],
       "layout": Layout(title="Predicted CLose Price - Random Forest  % ",
                         yaxis=dict(title='Close Price % (Daily) '),
                         xaxis=dict(title='Future Days Prediction Day the from Current Day - Test Data '))
})

In [80]:
def create_dataset1(datasetReal,i):
    print(type(datasetReal))
#     dataset = datasetReal.copy()
#     y_train = np.array(dataset['closePrice'])
#     y_train2 = y_train[2:]
    xdataset = datasetReal.copy()
#     del xdataset['closePrice']
    xdataset = xdataset[xdataset.columns[xdataset.columns !='closePrice']]
    dataset = np.array(xdataset)
    print("Total Number of features in trainig Dataset "+ str(xdataset.columns.size))
    dataX = [dataset[n] for n in range(len(dataset)-i)]
    return np.array(dataX),xdataset.columns.size

In [96]:
frameshistory=[]
for i in range(1,21):
    print("Training the Neural Network for next "+ str(i)+" Prediction")
    starttime= time.time() 
    inputdata = inputClusterone.copy()
    dataset1 = get_indicators(inputClusterone,i)
    
    print("Making The Training Dataset")
    traindataset = dataset1.head(len(dataset1)-50)
    
    trainY = np.array(traindataset['closePrice'])[i:]
    trainX, numbFeatures = create_dataset1(traindataset,i)
    
    m,h = stock_prediction(trainX,trainY)
    fullReport  = pd.DataFrame(h.history)
    fullReport.to_csv("Day_"+ str(i)+" NN_Prediction.csv")
    finalMAPE=fullReport['mean_absolute_percentage_error'].tail(1)
    finalLoss = fullReport['loss'].tail(1)
    finalMAE = fullReport['mean_absolute_error'].tail(1)

    finalreport = pd.DataFrame(finalMAPE)
    finalreport.columns = ['MAPE']
    finalreport['Loss']=finalLoss
    finalreport['MAE']=finalMAE
    finalreport.index = range(1,len(finalreport)+1)
    
    frameshistory.append(finalreport)
    print(finalreport)
    
    
    trainfeature=traindataset.tail(len(traindataset)-1)
    trainlabel= trainfeature['closePrice']

    prediction = m.predict(trainX)
    my_list = map(lambda x: x[0], prediction)
    traintestlabel = pd.Series(my_list)

    testdata = dataset1.tail(50)
    testexpectedlabel = testdata['closePrice']
    
    xtest = testdata[testdata.columns[testdata.columns !='closePrice']]
    predictiontest = m.predict(np.array(xtest))
    temp_list = map(lambda x: x[0], predictiontest)
    testlabel = pd.Series(temp_list)
    print("Time Taken to Model %s seconds ---" % (time.time() - starttime))

    iplot({"data": [Scatter(x=trainfeature.index, y=trainlabel,name=str('Expected Train Data'+str(i))),
                    Scatter(x=trainfeature.index, y=traintestlabel,name=str('Actual Train Data'+str(i))),
                    Scatter(x=testdata.index, y=testexpectedlabel,name=str('Expected Test Data'+str(i))),
                    Scatter(x=testdata.index, y=testlabel,name=str('Actual Test'+str(i)))],
           "layout": Layout(title="Predicted CLose Price - Neural Network  % ",
                             yaxis=dict(title='Close Price % (Daily) '),
                             xaxis=dict(title='Future Days Prediction Day the from Current Day '+str(i)+'- Test Data '))
    })

Training the Neural Network for next 1 Prediction
Adding Features: Started  
Making The Training Dataset
<class 'pandas.core.frame.DataFrame'>
Total Number of features in trainig Dataset 10
Time Taken to Model 18.0306839943 seconds ---
*******************************************
*******************************************
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_42 (Dense)             (None, 11)                66        
_________________________________________________________________
dense_43 (Dense)             (None, 11)                132       
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 12        
Total params: 265
Trainable params: 265
Non-trainable params: 0
__________________

Training the Neural Network for next 2 Prediction
Adding Features: Started  
Making The Training Dataset
<class 'pandas.core.frame.DataFrame'>
Total Number of features in trainig Dataset 10
Time Taken to Model 18.1802999973 seconds ---
*******************************************
*******************************************
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_46 (Dense)             (None, 11)                66        
_________________________________________________________________
dense_47 (Dense)             (None, 11)                132       
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 12        
Total params: 265
Trainable params: 265
Non-trainable params: 0
__________________

In [97]:
allrecord = pd.concat(frameshistory)
allrecord.index = range(1,len(allrecord)+1)

In [100]:
allrecord

,MAPE,Loss,MAE
1,1.511234,85854.580982,198.814118
2,1.896901,136054.104302,250.460444
